In [1]:
from neo4j import GraphDatabase
import os

In [2]:
AURA_CONNECTION_URI = os.getenv('NEO4J_URI')
AURA_USERNAME = os.getenv('NEO4J_USER')
AURA_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)

In [4]:
import json

with open('mock_data/food.json', 'r') as f:
    foods = json.load(f)

In [12]:
bag_of_words = {}
simple_ingredient = {}

for food, temp in foods.items():
    ingredients = temp['Ingredient']
    materials = [temp.split('(')[0].strip().lower() if '(' in temp else temp.strip().lower() for temp in ingredients]

    if food == "pad_thai": continue
    food = food.replace('_', ' ').lower()
    simple_ingredient[food] = materials
    
    for material in materials:
        if material in bag_of_words:
            bag_of_words[material] += 1
        else:
            bag_of_words[material] = 1

In [13]:
bag_of_words

{'mackerel': 1,
 'chili': 1,
 'garlic': 4,
 'shallots': 2,
 'shrimp paste': 1,
 'lime juice': 4,
 'vegetables': 1,
 'rice flour': 1,
 'coconut milk': 5,
 'sugar': 6,
 'salt': 2,
 'spring onions': 2,
 'chicken': 3,
 'turmeric powder': 1,
 'curry powder': 1,
 'peanut sauce': 1,
 'cucumber relish': 1,
 'fish paste': 1,
 'red curry paste': 1,
 'kaffir lime leaves': 3,
 'green beans': 1,
 'egg': 3,
 'fish sauce': 6,
 'grilled pork': 1,
 'chili powder': 1,
 'roasted rice powder': 1,
 'mint leaves': 1,
 'cilantro': 2,
 'wide rice noodles': 1,
 'soy sauce': 2,
 'chinese broccoli': 1,
 'pork or chicken': 1,
 'glutinous rice flour': 1,
 'palm sugar': 3,
 'grated coconut': 1,
 'water': 3,
 'soft tofu': 1,
 'ginger': 1,
 'massaman curry paste': 1,
 'beef': 1,
 'potatoes': 1,
 'onions': 1,
 'peanuts': 2,
 'cashew nuts': 1,
 'cinnamon': 1,
 'star anise': 1,
 'tamarind paste': 1,
 'shrimp': 1,
 'lemongrass': 1,
 'galangal': 1,
 "thai bird's eye chili": 2,
 'mushroom': 1,
 'green papaya': 1,
 'tamarin

In [14]:
similar_words = {}

for words in bag_of_words.keys():
    list_words = []
    for temp in bag_of_words.keys():
        if temp.lower() != words.lower() and words.lower() in temp.lower():
            list_words.append(temp.lower())
    
    if len(list_words) > 0:
        similar_words[words.lower()] = list_words

similar_words

{'chili': ['chili powder', "thai bird's eye chili"],
 'rice flour': ['glutinous rice flour'],
 'sugar': ['palm sugar'],
 'chicken': ['pork or chicken', 'chicken or shrimp'],
 'egg': ['thai eggplant'],
 'onions': ['spring onions'],
 'shrimp': ['shrimp paste', 'chicken or shrimp'],
 'tamarind': ['tamarind paste'],
 'onion': ['spring onions', 'onions'],
 'lime': ['lime juice', 'kaffir lime leaves'],
 'cucumber': ['cucumber relish']}

In [15]:
def add_food_with_ingredients(tx, food_name, ingredients):
    query = """
    CREATE (a:Food {name: $food_name})
    WITH a
    UNWIND $ingredients AS ingredient
    MERGE (b:Ingredient {name: ingredient}) 
    MERGE (b)-[:RECIPE]->(a)
    """
    tx.run(query, food_name=food_name, ingredients=ingredients)

# Iterating through food and ingredients
for food, ingredients in simple_ingredient.items():
    with driver.session() as session:
        session.write_transaction(add_food_with_ingredients, food, ingredients)

/tmp/ipykernel_15489/3250473487.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_food_with_ingredients, food, ingredients)


In [16]:
def add_ingredient_relations(tx, ingredient_term, ingredient_variants):
    query = """
    MATCH (a:Ingredient {name: $ingredient_term})
    UNWIND $ingredient_variants AS variant
    MERGE (b:Ingredient {name: variant})
    MERGE (a)-[:SYNONYM]->(b)  // Consider changing to a more appropriate relationship type if necessary
    """ 
    tx.run(query, ingredient_term=ingredient_term, ingredient_variants=ingredient_variants)

for ingredient_term, ingredient_variants in similar_words.items():
    with driver.session() as session:
        session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)

/tmp/ipykernel_15489/790334638.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)


In [ ]:
driver.close()